In [1]:
from TrendPy.webApp import App

In [2]:
a = App()

a.show()


HTML(value='<h3 align="center">Upload a .csv file here<h3>')

FigureWidget({
    'data': [{'mode': 'markers', 'name': 'datapoints', 'type': 'scatter', 'uid': '7dad9081-b371…